In [1]:
from util import *

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']
Connected DB CINS_SMY - conn
Connected oraDW_ANALYTICS - conn_aly


# Analyse Behavior (update continuously)

In [30]:
cohort06 = pd.read_pickle('data/COHORT_06')
cohort07 = pd.read_pickle('data/COHORT_07')
cohort08 = pd.read_pickle('data/COHORT_08')
cohort09 = pd.read_pickle('data/COHORT_09')
cohort10 = pd.read_pickle('data/COHORT_10')
cohort11 = pd.read_pickle('data/COHORT_11')
cohort12 = pd.read_pickle('data/COHORT_12')

In [31]:
k = 100000

Deploy lead Jun 2023

In [32]:
cust = []
x6 = cohort06.sort_values(by='SCORE', ascending=False)[:k]
cust.extend(x6['CUSTOMER_CDE'].tolist())
x7 = cohort07[~cohort07['CUSTOMER_CDE'].isin(cust)].sort_values(by='SCORE', ascending=False)[:k]
cust.extend(x7['CUSTOMER_CDE'].tolist())
x8 = cohort08[~cohort08['CUSTOMER_CDE'].isin(cust)].sort_values(by='SCORE', ascending=False)[:k]
cust.extend(x8['CUSTOMER_CDE'].tolist())
x9 = cohort09[~cohort09['CUSTOMER_CDE'].isin(cust)].sort_values(by='SCORE', ascending=False)[:k]
cust.extend(x9['CUSTOMER_CDE'].tolist())
x10 = cohort10[~cohort10['CUSTOMER_CDE'].isin(cust)].sort_values(by='SCORE', ascending=False)[:k]
cust.extend(x10['CUSTOMER_CDE'].tolist())
x11 = cohort11[~cohort11['CUSTOMER_CDE'].isin(cust)].sort_values(by='SCORE', ascending=False)[:k]
cust.extend(x11['CUSTOMER_CDE'].tolist())
x12 = cohort12[~cohort12['CUSTOMER_CDE'].isin(cust)].sort_values(by='SCORE', ascending=False)[:k]
cust.extend(x12['CUSTOMER_CDE'].tolist())
print(len(set(cust)))

700000


In [33]:
len(x6), len(x7), len(x8), len(x9), len(x10), len(x11), len(x12)

(100000, 100000, 100000, 100000, 100000, 100000, 100000)

In [1]:
set(x6['CUSTOMER_CDE']) & set(x7['CUSTOMER_CDE']), set(x6['CUSTOMER_CDE']) & set(x12['CUSTOMER_CDE'])

NameError: name 'x6' is not defined

In [35]:
x6['CUST_STT'].value_counts(), x7['CUST_STT'].value_counts(), x8['CUST_STT'].value_counts(), x9['CUST_STT'].value_counts(), x10['CUST_STT'].value_counts(), x11['CUST_STT'].value_counts(), x12['CUST_STT'].value_counts()

(2    34815
 0    33096
 1    32089
 Name: CUST_STT, dtype: int64,
 2    54214
 0    23524
 1    22262
 Name: CUST_STT, dtype: int64,
 0    40370
 1    34149
 2    25481
 Name: CUST_STT, dtype: int64,
 0    37344
 1    36290
 2    26366
 Name: CUST_STT, dtype: int64,
 0    39723
 2    32996
 1    27281
 Name: CUST_STT, dtype: int64,
 0    41150
 2    33202
 1    25648
 Name: CUST_STT, dtype: int64,
 2    71074
 1    15042
 0    13884
 Name: CUST_STT, dtype: int64)

In [36]:
def analyze_cohort(cohort):
    mob = cohort['MOB'].tolist()[0]
    len_cohort = len(cohort)
    data = []
    num_react = len(cohort[cohort['CUST_STT'] == 2])
    rpt_dt = cohort['RPT_DT'].tolist()[0]
    cohort = cohort[cohort['CUST_STT'] != 2]
    len_not_react = len(cohort)
    data.append((rpt_dt,num_react, len_not_react))
    if len(cohort) > 0:
        for i in range(1,10):
            if len(cohort) > 0:
                lead_mth = f'CUST_STT_LEAD{i}'
                num_react = len(cohort[cohort[lead_mth] == 2])
                rpt_dt = cohort[f'RPT_DT_LEAD{i}'].tolist()[0]
                cohort = cohort[cohort[lead_mth] != 2]
                len_not_react = len(cohort)
                data.append((rpt_dt,num_react, len_not_react))
    res_df = pd.DataFrame(data=data, columns=['RPT_DT', 'NUM_REACT', 'NUM_NOT_REACT'])
    res_df['PERC_REACT_PREV_MOB'] = res_df['NUM_REACT'].shift(-1) / res_df['NUM_NOT_REACT']
    res_df['PERC_REACT_PREV_MOB'] = res_df['PERC_REACT_PREV_MOB'].shift()
    res_df['CUMSUM'] = res_df['NUM_REACT'].cumsum()
    res_df['PERC'] = res_df['CUMSUM'] / len_cohort
    res_df = res_df[res_df['RPT_DT'].notnull()]
    res_df = res_df.reset_index()
    # ax = sns.barplot(res_df, x='RPT_DT', y='NUM_REACT')
    # sns.lineplot(res_df['CUMSUM'], ax=ax)
    # plt.show()
    res_df['MOB'] = mob
    res_df = res_df.set_index('MOB')
    return res_df

In [37]:
x6['MOB'] = '01-06-2023'
x7['MOB'] = '01-07-2023'
x8['MOB'] = '01-08-2023'
x9['MOB'] = '01-09-2023'
x10['MOB'] = '01-10-2023'
x11['MOB'] = '01-11-2023'
x12['MOB'] = '01-12-2023'

In [38]:
t = analyze_cohort(x6)
for x in [x7, x8, x9, x10, x11, x12]:
    t = pd.concat([t, analyze_cohort(x)])

In [39]:
t

,index,RPT_DT,NUM_REACT,NUM_NOT_REACT,PERC_REACT_PREV_MOB,CUMSUM,PERC
MOB,,,,,,,
01-06-2023,0,2023-05-31,34815,65185,NaN,34815,0.34815
01-06-2023,1,2023-06-30,5084,60101,0.077993,39899,0.39899
01-06-2023,2,2023-07-31,1903,58198,0.031663,41802,0.41802
01-06-2023,3,2023-08-31,1824,56374,0.031341,43626,0.43626
01-06-2023,4,2023-09-30,1610,54764,0.028559,45236,0.45236
01-06-2023,5,2023-10-31,1703,53061,0.031097,46939,0.46939
01-06-2023,6,2023-11-30,2369,50692,0.044647,49308,0.49308
01-06-2023,7,2023-12-31,966,49726,0.019056,50274,0.50274
01-07-2023,0,2023-06-30,54214,45786,NaN,54214,0.54214


In [40]:
t = analyze_cohort(x6)[['PERC']]
t.rename(columns={'PERC': 'MoB-6'}, inplace=True)
t = t.reset_index(drop=True)
p = 7
for x in [x7, x8, x9, x10, x11, x12]:
    z = analyze_cohort(x)[['PERC']]
    z.rename(columns={'PERC': f'Mob-{p}'}, inplace=True)
    z = z.reset_index(drop=True)
    t = pd.concat([t, z], axis=1)
    p += 1

In [41]:
t.T

,0,1,2,3,4,5,6,7
MoB-6,0.34815,0.39899,0.41802,0.43626,0.45236,0.46939,0.49308,0.50274
Mob-7,0.54214,0.56570,0.57970,0.59055,0.60126,0.61957,0.62553,NaN
Mob-8,0.25481,0.28888,0.30896,0.32710,0.35593,0.36611,NaN,NaN
Mob-9,0.26366,0.28522,0.30007,0.32497,0.33279,NaN,NaN,NaN
Mob-10,0.32996,0.35164,0.37034,0.37643,NaN,NaN,NaN,NaN
Mob-11,0.33202,0.36549,0.37362,NaN,NaN,NaN,NaN,NaN
Mob-12,0.71074,0.72897,NaN,NaN,NaN,NaN,NaN,NaN
